In [ ]:
import pandas as pd
import glob, io, requests

#two input files
location = "*.csv"
files = glob.glob(location)

output_file = 'v4-lms.csv'

rates_file = [file for file in files if "rates" in file][0]
levels_file = [file for file in files if "levels" in file][0]

df_rates = pd.read_csv(rates_file, dtype=str)
df_levels = pd.read_csv(levels_file, dtype=str)

'''Functions'''

def SeasonalValues(value):
    if value.startswith('Non'):
        return 'Non Seasonal Adjustment'
    else:
        return 'Seasonal Adjustment'
    
def SeasonalValueChanges(value):
    lookup = {
            'Non Seasonal Adjustment':'Not Seasonally Adjusted',
            'Seasonal Adjustment':'Seasonally Adjusted'
            }
    return lookup[value]

def SexLabel(value):
    # changes 'people' to 'all adults'
    lookup = {'people':'all-adults', 'People':'All adults'}
    return lookup.get(value, value)

def Slugize(value):
    new_value = value.replace(' ', '-').lower()
    return new_value

def Get_Latest_Version_From_CMD():
    '''
    Pulls the latest v4 from CMD for lms
    '''
    editions_url = 'https://api.beta.ons.gov.uk/v1/datasets/labour-market/editions/time-series/versions'
    r = requests.get(editions_url)
    if r.status_code != 200:
        logging_message = 'From Get_Latest_Version_From_CMD() - {} returned a {} error trying to obtain latest version number'.format(editions_url, r.status_code)
        raise Exception(logging_message)
    items = r.json()['items'] 

    # get latest version number
    latest_version_number = items[0]['version']
    if latest_version_number != len(items):
        logging_message = 'From Get_Latest_Version_From_CMD() - Number of versions "{}" does not match latest version number "{}"'.format(len(items), latest_version_number)
        raise Exception(logging_message)
        
    # get latest version data
    url = editions_url + '/' + str(latest_version_number)
    r = requests.get(url)
    if r.status_code == 200:
        latest_version = r.json()
        logging_message = 'CPIH version {} successfully pulled from the API'.format(latest_version_number)
        print(logging_message)
    else:
        logging_message = 'CPIH version {} not successfully pulled from API returned a {} error'.format(latest_version_number, r.status_code)
        raise Exception(logging_message)
    
    # decode data frame
    file_location = requests.get(latest_version['downloads']['csv']['href'])
    file_object = io.StringIO(file_location.content.decode('utf-8'))
    df = pd.read_csv(file_object, dtype=str)
    return df

#renaming the columns we are interested in
rename_cols = {
            'obs':'V4_0',
            'Time dim it id':'Time',
            'dim1itid':'EconomicActivity',
            'dim2itid':'AgeGroups',
            'dim3itid':'Sex',
            'dim4itid':'SeasonalAdjustment'
            }

df_rates = df_rates.rename(columns=rename_cols)
df_levels = df_levels.rename(columns=rename_cols)

#adding in codelist columns
df_rates['mmm-mmm-yyyy'] = df_rates['Time']
df_rates['uk-only'] = 'K02000001'
df_rates['Geography'] = 'United Kingdom'
df_rates['unit-of-measure'] = 'rates'
df_rates['UnitOfMeasure'] = 'Rates'
df_rates['economic-activity'] = df_rates['EconomicActivity']
df_rates['age-groups'] = df_rates['AgeGroups']
df_rates['seasonal-adjustment'] = df_rates['SeasonalAdjustment']

df_levels['mmm-mmm-yyyy'] = df_levels['Time']
df_levels['uk-only'] = 'K02000001'
df_levels['Geography'] = 'United Kingdom'
df_levels['unit-of-measure'] = 'levels'
df_levels['UnitOfMeasure'] = 'Levels'
df_levels['economic-activity'] = df_levels['EconomicActivity']
df_levels['age-groups'] = df_levels['AgeGroups']
df_levels['seasonal-adjustment'] = df_levels['SeasonalAdjustment']

df_rates = df_rates[[
        'V4_0', 'mmm-mmm-yyyy', 'Time', 'uk-only', 'Geography',
        'unit-of-measure', 'UnitOfMeasure', 'economic-activity','EconomicActivity', 
        'age-groups', 'AgeGroups', 'Sex', 'seasonal-adjustment', 'SeasonalAdjustment'
        ]]

df_levels = df_levels[[
        'V4_0', 'mmm-mmm-yyyy', 'Time', 'uk-only', 'Geography',
        'unit-of-measure', 'UnitOfMeasure', 'economic-activity','EconomicActivity', 
        'age-groups', 'AgeGroups', 'Sex', 'seasonal-adjustment', 'SeasonalAdjustment'
        ]]

'''Post Processing'''

df_rates['mmm-mmm-yyyy'] = df_rates['mmm-mmm-yyyy'].apply(Slugize)
df_levels['mmm-mmm-yyyy'] = df_levels['mmm-mmm-yyyy'].apply(Slugize)

df_rates['economic-activity'] = df_rates['economic-activity'].apply(Slugize)
df_levels['economic-activity'] = df_levels['economic-activity'].apply(Slugize)

df_rates['sex'] = df_rates['Sex'].apply(Slugize)
df_levels['sex'] = df_levels['Sex'].apply(Slugize)

df_rates['SeasonalAdjustment'] = df_rates['SeasonalAdjustment'].apply(SeasonalValues)
df_rates['seasonal-adjustment'] = df_rates['SeasonalAdjustment'].apply(Slugize)
df_rates['SeasonalAdjustment'] = df_rates['SeasonalAdjustment'].apply(SeasonalValueChanges)

df_levels['SeasonalAdjustment'] = df_levels['SeasonalAdjustment'].apply(SeasonalValues)
df_levels['seasonal-adjustment'] = df_levels['SeasonalAdjustment'].apply(Slugize)
df_levels['SeasonalAdjustment'] = df_levels['SeasonalAdjustment'].apply(SeasonalValueChanges)

#combining the two
df = pd.concat([df_rates, df_levels])

#moving data markings to a separate column
df.loc[df['V4_0'] == '*', 'Data Marking'] = '*'
df = df.rename(columns = {'V4_0':'V4_1'})
df = df[[
        'V4_1', 'Data Marking', 'mmm-mmm-yyyy', 'Time', 'uk-only', 'Geography',
        'unit-of-measure', 'UnitOfMeasure', 'economic-activity', 'EconomicActivity', 
        'age-groups', 'AgeGroups', 'sex', 'Sex', 'seasonal-adjustment', 'SeasonalAdjustment'
        ]]

df['sex'] = df['sex'].apply(SexLabel)
df['Sex'] = df['Sex'].apply(SexLabel)

previous_v4 = Get_Latest_Version_From_CMD()

new_df = pd.concat([previous_v4, df]).drop_duplicates()

new_df.to_csv(output_file, index=False)
print('Transform complete!')